 - Slicing 100k+ cells and ~4000 features from large (`~185 Gb`) .h5ad files was crashing the kernel. 
 - This was a problem in constructing the test dataset from the singleome data. 
 - Here we bypass the issue by creating files with 10k cells at a time, storing them to disk. 
 - The smaller files can then be loaded and concatenated into a single file that will be used downstream. 

In [1]:
from pathlib import Path
import warnings
import subprocess
import sys
import gc

import numpy as np
import pandas as pd
import anndata as ad
import scanpy as sc
from tqdm import tqdm
from rich import print as rprint
import pickle

from schelp.utils.config import load_config
from schelp.utils.data import donor_split, make_donor_splits_dataset

paths = load_config(dataset_key="init")

In [2]:
donor_frac_pergroup = 0.15

print("loading singleome data")
adata_singleome = ad.read_h5ad(paths["frozen"] / "SEAAD_MTG_RNAseq_Singleome_final-nuclei.2024-02-13.h5ad", backed="r")
train_ind, test_ind = donor_split(obs=adata_singleome.obs, donor_frac_pergroup=donor_frac_pergroup)

print("loading balanced data")
adata_balanced = ad.read_h5ad(
    str(paths["data"]) + "/Human-Brain/balanced_SEAAD_MTG_RNAseq_Singleome_final-nuclei.2024-06-18.h5ad"
)
# removing donors that are in the test set
test_donors = adata_singleome.obs["Donor ID"].loc[test_ind].unique()
adata_balanced = adata_balanced[~adata_balanced.obs["Donor ID"].isin(test_donors)]

loading singleome data


/allen/programs/celltypes/workgroups/mousecelltypes/Rohan/code/scgpt/schelp/schelp/utils/data.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_supertype_entropy = df.groupby("Donor ID")["Supertype"].apply(entropy).sort_values(ascending=False).to_frame()
/allen/programs/celltypes/workgroups/mousecelltypes/Rohan/code/scgpt/schelp/schelp/utils/data.py:45: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_.groupby("Overall AD neuropathological Change")[df_.columns]
/allen/programs/celltypes/workgroups/mousecelltypes/Rohan/code/scgpt/schelp/schelp/utils/data.py:51: FutureWarning: The default of observed=Fals

High supertype entropy donors make up 13.50% of total cells
Train set: 86.50%
Test set: 13.50%
loading balanced data


In [3]:
print("calculating variable genes")
# calculate variable genes based on training data
sc.pp.highly_variable_genes(adata_balanced, n_top_genes=4000, flavor="cell_ranger", batch_key="Donor ID")
adata_balanced = adata_balanced[:, adata_balanced.var["highly_variable"]]
gc.collect()

calculating variable genes


/allen/programs/celltypes/workgroups/mousecelltypes/Rohan/env/miniconda3/envs/scgpt/lib/python3.9/site-packages/scanpy/preprocessing/_highly_variable_genes.py:664: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns["hvg"] = {"flavor": flavor}


32107

In [4]:
# save adata_balanced to disk
adata_balanced.write(str(paths["data"]) + "/Human-Brain/v1_donor_split_train.h5ad")
print(f"Train data shape: {adata_balanced.shape}")
features = adata_balanced.var.index
del adata_balanced
gc.collect()

Train data shape: (118871, 4000)


7322

In [5]:
data_dict = dict(features=features, test_ind=test_ind)
with open(str(paths["data"]) + "/Human-Brain/v1_donor_split_intermediate_results.pkl", "wb") as f:
    pickle.dump(data_dict, f)
gc.collect()

0

In [6]:
with open(str(paths["data"]) + "/Human-Brain/v1_donor_split_intermediate_results.pkl", "rb") as f:
    data_dict_loaded = pickle.load(f)
print(data_dict_loaded.keys())

dict_keys(['features', 'test_ind'])


In [7]:
# load 10000 cells from test_ind at a time, same them to file and them move to the next 10000 cells till all test cells are saved
for i in tqdm(range(0, len(data_dict_loaded["test_ind"]), 10000)):
    print(f"Processing cells {i} to {i + 10000}")
    startt = i
    endd = np.min([i + 10000, len(data_dict_loaded["test_ind"])])
    temp_adata = adata_singleome[data_dict_loaded["test_ind"][startt:endd], :].to_memory()
    temp_adata = temp_adata[:, data_dict_loaded["features"]]
    temp_adata.write(f"temp_test_cells_{i}.h5ad")
    del temp_adata
    gc.collect()

  0%|          | 0/17 [00:00<?, ?it/s]

Processing cells 0 to 10000


  6%|▌         | 1/17 [00:35<09:26, 35.38s/it]

Processing cells 10000 to 20000


 12%|█▏        | 2/17 [01:12<09:06, 36.44s/it]

...

 88%|████████▊ | 15/17 [08:19<01:15, 37.51s/it]

Processing cells 150000 to 160000


 94%|█████████▍| 16/17 [08:56<00:37, 37.30s/it]

Processing cells 160000 to 170000


100%|██████████| 17/17 [09:19<00:00, 32.91s/it]


In [8]:
# load each temp file into memory and concatenate it into one file
adata_list = [None] * len(list(range(0, len(data_dict_loaded["test_ind"]), 10000)))
fnames_list = list(range(0, len(data_dict_loaded["test_ind"]), 10000))
for i, ff in tqdm(enumerate(fnames_list)):
    adata_list[i] = ad.read_h5ad(f"temp_test_cells_{ff}.h5ad")

adata = ad.concat(adata_list, axis=0)
adata.write(str(paths["data"]) + "/Human-Brain/v1_donor_split_test.h5ad")

del adata_list
gc.collect()

17it [00:54,  3.22s/it]


24440

In [9]:
display(adata)
display(adata.obs["Donor ID"].value_counts().to_frame())

AnnData object with n_obs × n_vars = 165668 × 4000
    obs: 'sample_id', 'Neurotypical reference', 'Donor ID', 'Organism', 'Brain Region', 'Sex', 'Gender', 'Age at Death', 'Race (choice=White)', 'Race (choice=Black/ African American)', 'Race (choice=Asian)', 'Race (choice=American Indian/ Alaska Native)', 'Race (choice=Native Hawaiian or Pacific Islander)', 'Race (choice=Unknown or unreported)', 'Race (choice=Other)', 'specify other race', 'Hispanic/Latino', 'Highest level of education', 'Years of education', 'PMI', 'Fresh Brain Weight', 'Brain pH', 'Overall AD neuropathological Change', 'Thal', 'Braak', 'CERAD score', 'Overall CAA Score', 'Highest Lewy Body Disease', 'Total Microinfarcts (not observed grossly)', 'Total microinfarcts in screening sections', 'Atherosclerosis', 'Arteriolosclerosis', 'LATE', 'Cognitive Status', 'Last CASI Score', 'Interval from last CASI in months', 'Last MMSE Score', 'Interval from last MMSE in months', 'Last MOCA Score', 'Interval from last MOCA in mont

,count
Donor ID,
H20.33.030,23676
H21.33.022,20263
H21.33.007,17056
H21.33.003,16127
H21.33.002,15761
H21.33.043,15005
H20.33.004,14537
H21.33.014,13836
H21.33.025,13608


In [10]:
# cleanup - removes all tempfiles
!rm temp_test_cells_*.h5ad